In [21]:
##VERSIÓN 1##
#En está versión voy a crear una única tabla a partir del csv

import csv
import mysql.connector

#he definido una función para conectarme al servidor y llamarlo desde el main
def connect_to_server():
    global mydb #he hecho esto global para que no se corté la conexión a lo largo del programa
    global mycursor
    mydb = mysql.connector.connect(
      host="dayhoff.inf.um.es",
      user="alubio07",
      password="alubio07PW-",
      database="alubio07"
    )
    mycursor = mydb.cursor()
    return mycursor

#he hecho está función para coger los headers del csv porque lo uso varias veces
def get_headers():
    with open("cod_amino.csv", mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",", )
        csv_headers = csv.DictReader(csv_file, delimiter=",", )
        fieldnames = csv_headers.fieldnames
        return fieldnames

def create_table():
    fieldnames = get_headers()
    mycursor.execute("CREATE TABLE cod_amino (\
      "+fieldnames[0]+" varchar(255) NOT NULL,\
      "+fieldnames[1]+" varchar(255) NOT NULL,\
      "+fieldnames[2]+" varchar(255) NOT NULL, \
      "+fieldnames[3]+" varchar(255) NOT NULL,\
      PRIMARY KEY ("+fieldnames[0]+")\
    )")
    
def charge_csv():
    with open("cod_amino.csv", mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",", )
        next(csv_reader)
        line_count = 0
        values = []
        for row in csv_reader:
            fila = (row[0], row[1], row[2], row[3])
            values.append(fila)
    #Creamos la plantilla de inserción
    fieldnames = get_headers()
    sql_insert = "INSERT INTO cod_amino ("+fieldnames[0]+", "+fieldnames[1]+", "+fieldnames[2]+", "+fieldnames[3]+") VALUES (%s, %s, %s, %s)"
    mycursor.executemany(sql_insert, values)
    mydb.commit()
    return mycursor.rowcount

def main():
    mycursor = connect_to_server()
    create_table()
    filas_insertadas = charge_csv()
    print(filas_insertadas, "filas insertadas")

if __name__ == "__main__":
    main()

64 filas insertadas


In [22]:
#con esto comprobamos que aparece la tabla que acabamos de crear en nuestra base de datos
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

('cod_amino',)
('departamento',)
('departamento_empleado',)
('empleado',)
('gasto',)
('genep2',)
('manager',)
('presupuesto',)
('proteinp2',)
('salario',)
('sequenceb3',)
('variantb3',)


In [20]:
#ejecutar esto para borrar la tabla en el caso de que se quiera comprobar que funciona el código de
#arriba, ya que si no saldrá un error de que ya existe la tabla
mycursor.execute("DROP TABLE cod_amino;")

In [23]:
##VERSIÓN 2##
#En esta versión voy a crear dos tablas que se relacionen entre sí a partir del csv

import csv
import mysql.connector

#he definido una función para conectarme al servidor y llamarlo desde el main
def connect_to_server():
    global mydb #he hecho esto global para que no se corté la conexión a lo largo del programa
    global mycursor
    mydb = mysql.connector.connect(
      host="dayhoff.inf.um.es",
      user="alubio07",
      password="alubio07PW-",
      database="alubio07"
    )
    mycursor = mydb.cursor()
    return mycursor

def get_headers():
    with open("cod_amino.csv", mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",", )
        csv_headers = csv.DictReader(csv_file, delimiter=",", )
        fieldnames = csv_headers.fieldnames
        return fieldnames

def create_table1():
    fieldnames = get_headers()
    mycursor.execute("CREATE TABLE cod_amino1 (\
      "+fieldnames[1]+" varchar(255) NOT NULL,\
      "+fieldnames[2]+" varchar(255) NOT NULL,\
      "+fieldnames[3]+" varchar(255) NOT NULL,\
      PRIMARY KEY ("+fieldnames[1]+")\
    )")

def create_table2():
    fieldnames = get_headers()
    mycursor.execute("CREATE TABLE cod_amino2 (\
      "+fieldnames[0]+" varchar(255) NOT NULL,\
      "+fieldnames[1]+" varchar(255) NOT NULL,\
      PRIMARY KEY ("+fieldnames[0]+"),\
      FOREIGN KEY ("+fieldnames[1]+") REFERENCES cod_amino1 ("+fieldnames[1]+")\
    )")
    
def charge_csv():
    with open("cod_amino.csv", mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",", )
        next(csv_reader)
        line_count = 0
        values1 = []
        values2 = []
        for row in csv_reader:
            fila1 = (row[1], row[2], row[3])
            fila2 = (row[0], row[1])
            values2.append(fila2)
            if fila1 not in values1: #en esta tabla saldrían duplicados así que los evito aquí
                values1.append(fila1) #Imprimimos la columna símbolo
    #Creamos la plantilla de inserción
    fieldnames = get_headers()
    sql_insert1 = "INSERT INTO cod_amino1 ("+fieldnames[1]+", "+fieldnames[2]+", "+fieldnames[3]+") VALUES (%s, %s, %s)"
    sql_insert2 = "INSERT INTO cod_amino2 ("+fieldnames[0]+", "+fieldnames[1]+") VALUES (%s, %s)"
    mycursor.executemany(sql_insert1, values1)
    mydb.commit()
    filas_insertadas1 = mycursor.rowcount
    mycursor.executemany(sql_insert2, values2)
    mydb.commit()
    filas_insertadas2 = mycursor.rowcount
    return filas_insertadas1, filas_insertadas2

def main():
    mycursor = connect_to_server()
    create_table1()
    create_table2()
    filas_insertadas = charge_csv()
    print(filas_insertadas[0], "filas insertadas en la primera tabla y", filas_insertadas[1], "filas insertadas en la segunda tabla.")

if __name__ == "__main__":
    main()

21 filas insertadas en la primera tabla y 64 filas insertadas en la segunda tabla.


In [24]:
#con esto comprobamos que aparece la tabla que acabamos de crear en nuestra base de datos
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

('cod_amino',)
('cod_amino1',)
('cod_amino2',)
('departamento',)
('departamento_empleado',)
('empleado',)
('gasto',)
('genep2',)
('manager',)
('presupuesto',)
('proteinp2',)
('salario',)
('sequenceb3',)
('variantb3',)


In [19]:
#ejecutar esto para borrar las tablas en el caso de que se quiera comprobar que funciona el código de
#arriba, ya que si no saldrá un error de que ya existe la tabla
mycursor.execute("DROP TABLE cod_amino2;")
mycursor.execute("DROP TABLE cod_amino1;")